# Transfer Learning usando lasagne - parte 2

Nesse tutorial, vamos utilizar CNNs treinadas na base ImageNet para outros problemas de classificação de imagens, usando transfer learning.

O objetivo é treinar um modelo para discriminar um subconjunto de 5 classes da base de dados Caltech 101 (http://www.vision.caltech.edu/Image_Datasets/Caltech101/). A base para esse exercício possui 325 imagens de tamanho 224x224x3. 

Na parte 2 desse tutorial, vamos utilizar o método de Finetunning descrito em [1]: 
 1. Vamos utilizar uma rede treinada na base ImageNet: https://github.com/Lasagne/Recipes/tree/master/modelzoo
 2. Vamos remover a última camada da rede, adicionar novas camadas e prosseguir com o treinamento do modelo


[1] Oquab, Maxime, et al. "Learning and transferring mid-level image representations using convolutional neural networks." [link](http://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Oquab_Learning_and_Transferring_2014_CVPR_paper.pdf)

In [ ]:
import cPickle
import lasagne
import theano.tensor as T
import theano
import numpy as np
from train import train_minibatch

%load_ext autoreload
%autoreload 2

In [ ]:
data = np.load('caltech_5classes.npz')
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']
classes = data['classes']

In [ ]:
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from vgg_cnn_s_cpu import build_model

In [ ]:
params = cPickle.load(open('vgg_cnn_s.pkl'))
mean_img = params['mean image']

def process_dataset(x):
    x = np.transpose(x, (0,3,1,2))  # Modifica dados para: exemplos x canais RGB x altura x largura
    x = x[:, ::-1]                  # Modifica canais de RGB para BGR
    
    x = x - mean_img
    return x

x_train = process_dataset(x_train)
x_valid = process_dataset(x_valid)
x_test = process_dataset(x_test)

In [ ]:
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import softmax

def build_model_for_finetuning(params):
    model = build_model()
    
    lasagne.layers.set_all_param_values(model['prob'], params['values'])
    del model['fc8']
    del model['prob']
    model['fc8'] = DenseLayer(model['drop7'], 5, nonlinearity=softmax )

    model['data'] = model['input']
    model['out'] = model['fc8']
    return model

In [ ]:
def get_output_batch(function, x, batch_size=32):
    output = []
    for batch_start in xrange(0, len(x), batch_size):
        output.append(function(x[batch_start:batch_start+batch_size]))
    return np.vstack(output)



In [ ]:
from lasagne.regularization import l2, regularize_layer_params

In [ ]:
def compile_train_function(net, lr, w_decay):
    input_var = net['data'].input_var
    output_var = T.ivector()

    predicted = lasagne.layers.get_output(net['out'], inputs=input_var)
    loss = lasagne.objectives.categorical_crossentropy(predicted, output_var)
    loss = loss.mean()
    loss += w_decay * regularize_layer_params(net['out'], l2)
    

    y_pred = T.argmax(predicted, axis=1)
    acc = T.eq(y_pred, output_var)
    acc = acc.mean()

    params = lasagne.layers.get_all_params(net['out'])
    updates = lasagne.updates.sgd(loss, params, lr)

    train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
    val_fn = theano.function([input_var, output_var], [loss, acc])
    return train_fn, val_fn

In [ ]:
model = build_model_for_finetuning(params)

train_fn, valid_fn = compile_train_function(model, lr=0.001, w_decay= 1e-4)

In [ ]:
train_curves = train_minibatch(train_fn, valid_fn,     # Treinamento usando Batch Gradient Descent
                     train_set=(x_train, y_train), 
                     valid_set=(x_test, y_test),
                     epochs=10,
                     batch_size=16)


In [ ]:
print 'Train accuracy: ', train_curves[1][-1]
print 'Val accuracy: ', train_curves[3][-1]

In [ ]:



def compile_train_function_somelayers(net, lr, w_decay, layers_to_train):
    input_var = net['data'].input_var
    output_var = T.ivector()

    predicted = lasagne.layers.get_output(net['out'], inputs=input_var)
    loss = lasagne.objectives.categorical_crossentropy(predicted, output_var)
    loss = loss.mean()
    loss += w_decay * regularize_layer_params(net['out'], l2)
    

    y_pred = T.argmax(predicted, axis=1)
    acc = T.eq(y_pred, output_var)
    acc = acc.mean()

    params = []
    for l in layers_to_train:
        params += l.get_params(trainable=True)
    updates = lasagne.updates.sgd(loss, params, lr)

    train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
    val_fn = theano.function([input_var, output_var], [loss, acc])
    return train_fn, val_fn

In [ ]:
model = build_model_for_finetuning(params)

train_fn, valid_fn = compile_train_function_somelayers(model, lr=0.01, w_decay=1e-4, layers_to_train=[model['fc8']])

In [ ]:
#Liberar memoria não utilizada:

import gc
gc.collect()

In [ ]:
train_curves = train_minibatch(train_fn, valid_fn,     # Treinamento usando Batch Gradient Descent
                     train_set=(x_train, y_train), 
                     valid_set=(x_test, y_test),
                     epochs=10,
                     batch_size=16)


In [ ]:
def get_valid_acc_batch(function, x, y, batch_size=32):
    acclist = []
    for batch_start in xrange(0, len(x), batch_size):
        acclist.append(function(x[batch_start:batch_start+batch_size], y[batch_start:batch_start+batch_size]))
    return np.mean(acclist)



In [ ]:
get_valid_acc_batch(valid_fn, x_train, y_train)

In [ ]:
valid_fn(x_valid, y_valid)

In [ ]:
import sys

sys.stdout.write('a')
sys.stdout.write('a')
